In [ ]:
import torch
import os
import yaml
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from models.model import DPTAffordanceModel
import utils.transform as TF

torch.set_grad_enabled(False)

In [ ]:
split_mode = "object" # object or actor

dataset_root_path = ""

resume = ""

In [ ]:
affordance = ["openable", "cuttable", "pourable", "containable", "supportable", "holdable"]
num_affordance = len(affordance)
num_object = 12

In [ ]:
keypoint_path = os.path.join(dataset_root_path, split_mode, "train_affordance_keypoint.yaml")
with open(keypoint_path, "r") as fb:
    keypoint_dict = yaml.safe_load(fb)

In [ ]:
model = DPTAffordanceModel(num_object, num_affordance, use_hf=True).cuda()
ckpt = torch.load(resume, map_location=lambda storage, loc: storage)
model.load_state_dict(
    {
        k.replace("module.", ""): v
        for k, v in ckpt["state_dict"].items()
    },
    strict=False,
)
model.eval()

In [ ]:
if split_mode == "object":
    mean = [132.2723, 106.8666, 112.8962]
    std = [67.4025, 70.7446, 72.1553]
elif split_mode == "actor":
    mean = [136.5133, 108.5417, 113.0168]
    std = [67.4025, 70.7446, 72.1553]
else:
    raise Exception(f"split_mode: {split_mode} is not supported.")

tf = TF.Compose(
    [
        TF.PILToTensor(),
        TF.ImageNormalizeTensor(mean=mean, std=std),
    ]
)

In [ ]:
file_name = "10001_1"

In [ ]:
file_path = os.path.join(dataset_root_path, split_mode, "images", f"{file_name}.png")
image = Image.open(file_path)

data = {
    "file_name": file_name,
    "image": image,
}

data_tf = tf(data)

input = data_tf["image"].unsqueeze(0).cuda()
output_list = model(input)

In [ ]:
image_bgr = np.array(image)[:, :, ::-1]

fig, axes = plt.subplots(1, num_affordance)
fig.set_dpi(300)
fig.set_figheight(2)
fig.set_figwidth(2 * num_affordance)
for idx, it in enumerate(output_list):
    it = (it > 0).cpu().numpy().astype(np.uint8).squeeze()
    mask = np.zeros((it.shape[0], it.shape[1], 3), dtype=np.uint8)
    mask[it==1] = (255, 255, 0)
    image_mask = cv2.addWeighted(image_bgr, 1.0, mask, 0.8, 0.0)
    image_mask = image_mask[:, :, ::-1]

    axes[idx].imshow(image_mask)
    axes[idx].axis("off")
    axes[idx].set_title(f"{affordance[idx]}", fontsize=12)
fig.tight_layout(pad=0)
fig.show()